In [ ]:
# Import modules
import pyodbc
import pandas as pd
import time
from datetime import datetime

In [ ]:
# Connect to SQL Server
conn = pyodbc.connect('Driver={ODBC Driver 13 for SQL Server};'
                     'Server=server_name.com;'
                     'Database=database_name;'
                     'Trusted_Connection=no;'
                     'ApplicationIntent=READONLY;'
                     'UID=uid_name;'
                     'PWD=abcdefghijk')

In [ ]:
# Query list of all assets
start = time.time()

sqlString =     """
                SELECT
                    tblInventory.code AS asset_code,
                    tblInventory.as_id,
                    tblValues.current_value
                    
                FROM (tblInventory
                /*Status Change*/
                LEFT JOIN tblValues
                    ON (tblInventory.id = tblValues.id
                        AND (tblValues.field_id = 100250)))
                
                WHERE tblInventory.ab_id = 1
                    AND (LEFT(tblInventory.code, 1) IN('0','1','2'))
                    AND (tblInventory.code <> '0')
                ORDER BY asset_code ASC
                """

# Read SQL into dataframe
assets_df = pd.read_sql(sqlString, conn)

end = time.time()
print('Time elapsed = ' + str(end - start))

# Verify the query columns and shape
print(assets_df.head(5))
print('Number of records = ' + str(assets_df.shape[0]))
print('Number of columns = ' + str(assets_df.shape[1]))

In [ ]:
# Query asset values from change log based on date
sqlString =     """
                SELECT
                    tblInventoryTasks.as_id,
                    tblValueLog.c_value AS current_value,
                    tblValueLog.o_value AS old_value,
                    tblValueLog.value_date AS date_changed
                    
                FROM (tblInventoryTasks
                /*Current Value*/
                LEFT JOIN tblValueLog
                    ON (tblInventoryTasks.as_id = tblValues.as_id
                        AND (tblValueLog.field_id = 300650)))
                
                WHERE tblValueLog.value_date > Convert(datetime, '2021-12-25')
                    AND ((tblValueLog.c_value IN('P','B','K'))
                        OR (tblValueLog.o_value IN('P','B','K'))
                ORDER BY date_changed ASC
                """

# Read SQL into dataframe
df = pd.read_sql(sqlString, conn)
print(df)

In [ ]:
# Merge both dataframes
df = df.merge(assets_df, how='left', left_on='as_id', right_on='as_id')

# Confirm merge worked correctly
print(df.head(50))
print('Number of records = ' + str(df.shape[0]))
print('Number of columns = ' + str(df.shape[1]))

In [ ]:
# Create function for finding new and old changes
def new_old_pbk(row):
    if row['current_value'] == 'P':
        if row['old_value'] == 'P':
            return None
        elif row['old_value'] == 'B':
            return None
        elif row['old_value'] == 'K':
            return None
        else:
            return 'New'
    elif row['current_value'] == 'B':
        if row['old_value'] == 'P':
            return None
        elif row['old_value'] == 'B':
            return None
        elif row['old_value'] == 'K':
            return None
        else:
            return 'New'
    elif row['current_value'] == 'K':
        if row['old_value'] == 'P':
            return None
        elif row['old_value'] == 'B':
            return None
        elif row['old_value'] == 'K':
            return None
        else:
            return 'New'
    else:
        return 'Old'
    
# Apply function to dataframe and drop column
df['pbk_description'] = df.apply(new_old_pbk, axis=1)
df.drop(columns=['date_changed'], axis=1, inplace=True)

# Confirm function was successful
print(df.head(50))
print('Number of records = ' + str(df.shape[0]))
print('Number of columns = ' + str(df.shape[1]))

In [ ]:
# Export dataframe to excel
now = datetime.today()
datestamp = datetime(now.year, now.month, now.day)strftime('%m-%d-%Y')

xlFileName = r'C:\data_exports\export_' + datestamp + '.xlsx'

# Initialize pandas excel file writer (using openpyxl)
writer = pd.ExcelWriter(xlFileName, engine='openpyxl')

# Custom export function setting
def paste_to_sheet(df, new_sheet_name):
    df.to_excel(writer,
        sheet_name=new_sheet_name,
        index=True
        freeze_panes=(1,0),
        verbose=True)
    
# Convert dataframe to xlsxWriter Excel object
paste_to_sheet(df, 'Changed Data')

# Close Pandas Excel writer and output the Excel file
writer.save()
conn.close()
print('Output created')